# 4.0 Stacks and Queues

## 4.1 Implement a max stack

### Problem Statement
Implement a stack which adds a constant time `max()` operation to the standard stack interface of `push()` and `pop()`.

In [1]:
import collections
import unittest


class MaxStack(object):
    """Stack with support for constant time access to maximum element."""

    def __init__(self):
        self.stack = collections.deque()
    
    def push(self, x):
        # Entries in the stack record both element and max so far.
        maxx = max(x, self.stack[-1][1]) if len(self.stack) > 0 else x
        self.stack.append((x, maxx))  # Stack holds tuple of (x, maxx).

    def pop(self):
        if len(self.stack) > 0:
            return self.stack.pop()[0]
        raise IndexError('pop invalid: empty stack')
    
    def peek(self):
        if len(self.stack) > 0:
            return self.stack[-1][0]
        raise IndexError('peek invalid: empty stack')
        
    def max(self):
        if len(self.stack) > 0:
            return self.stack[-1][1]
        raise IndexError('max invald: empty stack')


class MaxStackTest(unittest.TestCase):

    def test_max_stack(self):
        s = MaxStack()

        # Check invalid operations on empty stack.
        self.assertRaises(IndexError, s.pop)
        self.assertRaises(IndexError, s.peek)
        self.assertRaises(IndexError, s.max)
        
        s.push(2)
        self.assertEqual(s.peek(), 2)
        self.assertEqual(s.max(), 2)
        
        s.push(1)
        self.assertEqual(s.peek(), 1)
        self.assertEqual(s.max(), 2)
        
        s.push(3)
        self.assertEqual(s.peek(), 3)
        self.assertEqual(s.max(), 3)
        
        self.assertEqual(s.pop(), 3)
        self.assertEqual(s.peek(), 1)
        self.assertEqual(s.max(), 2)
        
        self.assertEqual(s.pop(), 1)
        self.assertEqual(s.peek(), 2)
        self.assertEqual(s.max(), 2)

        self.assertEqual(s.pop(), 2)

        # Check invalid operations on empty stack.
        self.assertRaises(IndexError, s.pop)
        self.assertRaises(IndexError, s.peek)
        self.assertRaises(IndexError, s.max)


unittest.main(MaxStackTest(), argv=[''], verbosity=2, exit=False)

test_max_stack (__main__.MaxStackTest) ... ok

----------------------------------------------------------------------
Ran 1 test in 0.002s

OK


## 4.2 Determine whether braces are balanced

### Problem Statement
Given a string consisting of any of the following braces `{},(),[]` returns `True` when the braces are balanced and `False` otherwise.

#### Clarification
In order for a string to be considered balanced, all substrings must be balanced.  As a result, a string such as `([{]})` would not be considered balanced since the substring `[{]` is unbalanced.

In [2]:
import collections                                               
import unittest                              


# Index of closing braces mapped to expected opening brace.
CLOSING_BRACES = {'}':'{', ')':'(', ']':'['}

                                                                                                                                      
def balanced_braces(string):       
    """Return True when braces in string are balanced."""
    stack = collections.deque()
    for c in list(string):
        if c in CLOSING_BRACES:
            if len(stack) < 1:
                return False
            if stack[-1] is not CLOSING_BRACES[c]:
                return False
            stack.pop()
        else:
            stack.append(c)
    return len(stack) == 0

 
class BalancedBracesTest(unittest.TestCase):

    def test_balanced_braces(self):
        case = collections.namedtuple('case', ['input','expected'])
        cases = [
            case('{}', True),
            case('{', False),  # Missing closing brace.
            case(']', False),  # Missing opening brace.
            case('{]', False),  # Brace mismatch.
            case('{[()]}', True),
            case('{[(])}', False), # Internal brace mismatch.
            case('{{[[(())]]}}', True),
        ]
        for c in cases:
            rcv = balanced_braces(c.input)
            self.assertEqual(rcv, c.expected)
                                     
                                                
unittest.main(BalancedBracesTest(), argv=[''], verbosity=2, exit=False)

test_balanced_braces (__main__.BalancedBracesTest) ... ok

----------------------------------------------------------------------
Ran 1 test in 0.011s

OK


## 4.3 Compute maximum of k-length subarrays

### Problem Statement
Given an array of integers of size $n$ and value $k$ such that $1 \leq k \leq n$, return an array containing the maximum element from each of the k-length subarrays of the input array.

#### Variation A
Use $k$ additional storage.

In [3]:
import collections
import unittest


class MaxQueue(object):
    """Queue with support for amortized constant time access to max."""

    def __init__(self):
        self.queue = collections.deque()
        self.max_queue = collections.deque()

    def enqueue(self, x):
        # Iteratively remove elements from max_queue smaller than current.
        while len(self.max_queue) > 0 and x > self.max_queue[-1]:
            self.max_queue.pop()
        self.max_queue.append(x)
        self.queue.append(x)

    def dequeue(self):
        if len(self.queue) > 0:
            x = self.queue.popleft()
            if x == self.max_queue[0]:
                self.max_queue.popleft()
            return x
        raise IndexError('dequeue invalid: empty queue')

    def max(self):
        if len(self.max_queue) > 0:
            return self.max_queue[0]
        raise IndexError('max invalid: empty queue')


def max_k_subarrays_varA(elems, k):       
    """Return an array containing the max of the k-length subarrays."""
    assert not(len(elems) < k), 'invalid: len(elems) < k'

    # Fill max queue with sliding window of k elements.
    queue = MaxQueue()
    for ind in range(len(elems)):
        if ind >= k:
            elems[ind-k] = queue.max()
            queue.dequeue()
        queue.enqueue(elems[ind])
    elems[len(elems)-k] = queue.max()
    return elems[:len(elems)-k+1]


class MaxKSubarraysTest(unittest.TestCase):
 
    def test_max_k_subarrays_varA(self):
        case = collections.namedtuple('case', ['input','k','expected'])
        cases = [
            case([10,5,2,7,8,7],2, [10,5,7,8,8]),
            case([10,5,2,7,8,7],3, [10,7,8,8]),
            case([10,5,2,7,8,7],4, [10,8,8]),
            case([10,5,2,7,8,7],5, [10,8]),
        ]
        for c in cases:
            rcv = max_k_subarrays_varA(c.input, c.k)
            self.assertEqual(rcv, c.expected)


unittest.main(MaxKSubarraysTest(), argv=[''], verbosity=2, exit=False)

test_max_k_subarrays_varA (__main__.MaxKSubarraysTest) ... ok

----------------------------------------------------------------------
Ran 1 test in 0.003s

OK


#### Variation B
Use no additional storage.

In [4]:
import collections                                               
import unittest                              
                                             
                                                                                                                                      
def max_k_subarrays_varB(elems, k):       
    """Return an array containing the max of the k-length subarrays."""
    assert not(len(elems) < k), 'invalid: len(elems) < k'

    # Make k passes through array finding max of 2-element subarrays.
    i0, iN = 0, len(elems)
    for _ in range(k-1):
        # Find the maximum of each pair of elements from i0..iN.
        ind1, ind2 = i0, i0+1
        while ind2 < iN:
            elems[ind1] = max(elems[ind1], elems[ind2])
            ind1, ind2 = ind1+1, ind2+1
        # On each pass the array of elements to compare shrinks by 1.
        iN -= 1
    return elems[i0:iN]


class MaxKSubarraysTest(unittest.TestCase):
 
    def test_max_k_subarrays_varB(self):
        case = collections.namedtuple('case', ['input','k','expected'])
        cases = [
            case([10,5,2,7,8,7],2, [10,5,7,8,8]),
            case([10,5,2,7,8,7],3, [10,7,8,8]),
            case([10,5,2,7,8,7],4, [10,8,8]),
            case([10,5,2,7,8,7],5, [10,8]),
        ]
        for c in cases:
            rcv = max_k_subarrays_varB(c.input, c.k)
            self.assertEqual(rcv, c.expected)
                                     
                                                
unittest.main(MaxKSubarraysTest(), argv=[''], verbosity=2, exit=False)

test_max_k_subarrays_varB (__main__.MaxKSubarraysTest) ... ok

----------------------------------------------------------------------
Ran 1 test in 0.003s

OK


## 4.4 Reconstruct array using +/- signs

### Problem Statement
The sequence `0..N` has been shuffled and the only clue you have about the true order is an array of `+/-` signs which indicate whether the element in that position of the array is larger or smaller than the previous element.  There is no signal for the first element.

Implement a function which reconstructs the true order from `+/-` signs.